In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as kr
import os, sys

In [19]:
import utils

In [20]:
#Loading the data
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train.shape, test.shape

((42000, 785), (28000, 784))

In [21]:
#Preparing the data
x_train = train.drop(["label"], axis=1).astype("float32")
y_train = train["label"].astype("int32")
x_test = test.astype("float32")
x_train = (x_train.values.reshape(-1, 28, 28, 1))/255.0
x_test = (x_test.values.reshape(-1, 28, 28, 1))/255.0
y_train = kr.utils.to_categorical(y_train, 10)

In [22]:
x_train.shape, x_test.shape, y_train.shape

((42000, 28, 28, 1), (28000, 28, 28, 1), (42000, 10))

In [61]:
model = kr.models.Sequential([
    kr.layers.InputLayer((28, 28, 1)),
    kr.layers.Conv2D(32, (3, 3), activation="relu"),
    kr.layers.Conv2D(32, (3, 3), activation="relu"),
    kr.layers.MaxPooling2D(),
    kr.layers.Dropout(0.25),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.MaxPooling2D(),
    kr.layers.Dropout(0.50),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.MaxPooling2D(),
    kr.layers.Dropout(0.25),
    kr.layers.Flatten(),
    kr.layers.Dense(10, activation="softmax")
])


model.summary()

optimizer = kr.optimizers.Adam(
    learning_rate= 0.001,
    beta_1= 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = "Adam"
)

model.compile(
    loss = "categorical_crossentropy",
    optimizer = optimizer,
    metrics = ["accuracy"]
)

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_357 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_358 (Conv2D)          (None, 24, 24, 32)        9248      
_________________________________________________________________
conv2d_359 (Conv2D)          (None, 22, 22, 32)        9248      
_________________________________________________________________
conv2d_360 (Conv2D)          (None, 20, 20, 32)        9248      
_________________________________________________________________
max_pooling2d_133 (MaxPoolin (None, 10, 10, 32)        0         
_________________________________________________________________
dropout_97 (Dropout)         (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_361 (Conv2D)          (None, 10, 10, 64)      

In [45]:
print("Shapes:  ", y_train.shape, model.output_shape)
print("Model complexity:  ", model.count_params())

Shapes:   (42000, 10) (None, 10)
Model complexity:   545514


In [62]:
model.fit(x = x_train, y = y_train, batch_size=50, epochs = 12)

Epoch 1/12
840/840 [==============================] - 336s 398ms/step - loss: 0.7841 - accuracy: 0.7146
Epoch 2/12
840/840 [==============================] - 366s 436ms/step - loss: 0.1387 - accuracy: 0.9640
Epoch 3/12
840/840 [==============================] - 356s 424ms/step - loss: 0.0924 - accuracy: 0.9758
Epoch 4/12
840/840 [==============================] - 371s 442ms/step - loss: 0.0821 - accuracy: 0.9788
Epoch 5/12
840/840 [==============================] - 351s 417ms/step - loss: 0.0747 - accuracy: 0.9815
Epoch 6/12
840/840 [==============================] - 353s 420ms/step - loss: 0.0643 - accuracy: 0.9836
Epoch 7/12
840/840 [==============================] - 364s 433ms/step - loss: 0.0621 - accuracy: 0.9848
Epoch 8/12
840/840 [==============================] - 361s 430ms/step - loss: 0.0575 - accuracy: 0.9855
Epoch 9/12
840/840 [==============================] - 348s 414ms/step - loss: 0.0512 - accuracy: 0.9870
Epoch 10/12
840/840 [==============================] - 350s 417m

In [63]:
results = model.predict(x_test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name="Label")
image_id = pd.Series(range(1, 28001), name = "ImageId")
submission = pd.concat([image_id, results], axis = 1)
submission.to_csv("../output/practice4.csv", index=False)